In [1]:
import cv2
import os
from tqdm import tqdm
import numpy as np
from scipy.signal import convolve2d

In [2]:
# save image
# clear all files in current folder


def save_tempimg(image):
    '''
    先清空当前文件夹，然后保存图片到temp_img文件夹下
    input: 
        a cv2 format image
    output:
        None (save image to temp_img/)
    '''
    all_files = os.listdir("temp_img/") # 获取目录下所有文件和目录

    # 遍历所有文件和子目录
    for file_or_dir in all_files:
        # 构建完整的文件或目录路径
        file_or_dir_path = os.path.join("temp_img/", file_or_dir)
        
        # 检查是否是文件并删除
        if os.path.isfile(file_or_dir_path):
            os.remove(file_or_dir_path) # 删除temp_img文件夹下的所有文件

    cv2.imwrite('temp_img/temp.png', image) # 保存 temp.png

def save_tempimg2(image):
    '''
    先清空当前文件夹，然后保存temp2图片到temp_img文件夹下
    input: 
        a cv2 format image
    output:
        None (save image to temp_img/)
    '''
    cv2.imwrite('temp_img/temp2.png', image)

def generate_padding(image, border_size=1):
    '''
    为图像添加padding
    input: 
        image: a cv2 format image
        border_size: int, padding size
    output:
        padded_image: a cv2 format image
    '''
    # Get the dimensions of the original image
    height, width = image.shape

    # Create a new image with the desired size
    padded_image = np.zeros((height + 2 * border_size, width + 2 * border_size), dtype=np.uint8)

    # Copy the original image to the center of the new image
    padded_image[border_size:border_size+height, border_size:border_size+width] = image

    return padded_image

# 读取图片

## 图片路径

In [3]:
from IPython.display import Image, display
image = cv2.imread('4.jpg', cv2.IMREAD_GRAYSCALE) # 更改'4.jpg'以切换图片路径

# Brightness Difference Processing 亮度差处理

In [4]:
image_test1 = image.copy()              #复制一份原图像用于处理
image_test2 = generate_padding(image)   #为图像添加padding

# image_test1 为原图
# image_test2 为添加padding后的图片

threshold_difference = 4
for i in tqdm(range(len(image_test1))):     #原图的每一行
    for j in range(len(image_test1[i])):    #原图的每一列
        current = int(image_test2[i][j])    #正在处理的中心像素亮度值
        top = int(image_test2[i-1][j])      #上方像素亮度值
        bot = int(image_test2[i+1][j])    #下方像素亮度值
        left = int(image_test2[i][j-1])     #左方像素亮度值
        right = int(image_test2[i][j+1])    #右方像素亮度值
        top_diff = abs(current - top)    #中心像素与上方像素亮度差
        bot_diff = abs(current - bot)   #中心像素与下方像素亮度差
        left_diff = abs(current - left)   #中心像素与左方像素亮度差
        right_diff = abs(current - right)   #中心像素与右方像素亮度差
        diff = max(top_diff, bot_diff, left_diff, right_diff)   #取最大差值
        if diff > threshold_difference or image_test1[i][j] <= 130:     #如果最大差值大于阈值或者中心像素亮度值小于等于130
            image_test1[i][j] = 0   #中心像素亮度值设为0
        if image_test1[i][j] > 150:
            image_test1[i][j] = 255 #否则中心像素亮度值设为255

  0%|          | 8/3024 [00:00<00:37, 79.44it/s]

100%|██████████| 3024/3024 [00:40<00:00, 75.36it/s]


In [6]:
save_tempimg(image_test1) # 保存brightness difference processed image为temp.png

# Imaging Denoising

Density measure
- 1. kernal size
- 2. black density
- Will use test_image_2 for paddling

用n*n的kernel划过每一行每一列，如果在image2中这一个pixel作为中心的kernel中的黑色像素比例超过threshold%，该像素在image1中将被替换成黑色，反之则会被替换为白色

此处的可替换variable为threshold数值

In [7]:
kernel_n = 9 # must be odd
threshold = 0.4                             # Adjustable threshold value
kernel = np.ones((kernel_n, kernel_n))
result_array = convolve2d(image_test1, kernel, mode='same', boundary='fill', fillvalue=0)
# multiply result_array by 1/225
white_pixel_count = result_array / 255
black_pixel_ratio = (81 - white_pixel_count) / 81

In [8]:
image_test3 = image_test1.copy()
for i in tqdm(range(len(image_test3))):
    for j in range(len(image_test3[i])):
        if black_pixel_ratio[i][j] < threshold:
            image_test3[i][j] = 255
        else:
            image_test3[i][j] = 0

  0%|          | 0/3024 [00:00<?, ?it/s]

100%|██████████| 3024/3024 [00:03<00:00, 872.60it/s]


In [9]:
save_tempimg2(image_test3)

In [10]:
# kernel_n = 9
# border_size = kernel_n // 2
# threshold = 0.4

# # test_image1 is the previously processed image, is a 2D numpy array contain only 0 and 255
# # shape of test_image1 is (height, width)

# # test_image2 is the padded image of test_image1, with a padding added to each side
# # and the size of the padding is border_size. shape of test_image2 is (height + 2 * border_size, width + 2 * border_size)

# # A kernel size with kernel_n will be used. This kernel will slide through each row and column of test_image2.
# # For each pixel, the kernel will be centered on it, and the percentage of black pixels in the kernel will be calculated.
# # If the percentage is greater than threshold, the pixel will be set to 0, otherwise it will be set to 255.
# test_image_1 = image_test.copy()
# test_image_2 = generate_padding(test_image_1, border_size=border_size)
# for i in tqdm(range(len(image_test))):
#     for j in range(len(image_test[i])):
#         y = i + border_size
#         x = j + border_size
#         kernel = test_image_2[y-border_size:y+border_size+1, x-border_size:x+border_size+1]
#         black_pixel_count = 0
#         for row in kernel:
#             for element in row:
#                 if element == 0:
#                     black_pixel_count += 1
#         black_percentage = black_pixel_count / (kernel_n * kernel_n)
#         if black_percentage > threshold:
#             test_image_1[i][j] = 0
#         else:
#             test_image_1[i][j] = 255


In [11]:
# save_tempimg2(test_image_1)

In [12]:
                #j   0  1  2  3  4      # i
k_array = np.array([[1 ,2 ,3 ,4 ,5],    # 0
                    [6 ,7 ,8 ,9 ,10],   # 1
                    [11,12,13,14,15],   # 2
                    [16,17,18,19,20]])  # 3

                    #x    0   1   2   3   4   5   6     # y
k_paddedled = np.array([[ 0,  0,  0,  0,  0,  0,  0],   # 0 # Can be generated using generate_padding(k_array, border_size=1)
                        [ 0,  1,  2,  3,  4,  5,  0],   # 1
                        [ 0,  6,  7,  8,  9, 10,  0],   # 2
                        [ 0, 11, 12, 13, 14, 15,  0],   # 3
                        [ 0, 16, 17, 18, 19, 20,  0],   # 4
                        [ 0,  0,  0,  0,  0,  0,  0]])  # 5

# Currently , i = 2, j = 1, looking at k_array[2][1] = 12
# As stated, y = i + border_size = 2 + 1 = 3
# and x = j + border_size = 1 + 1 = 2
# k_paddedled[3][2] = 12

In [13]:
k_paddedled[2:5, 1:4]

# here, we can see y is from 2 to 5, containing 3 rows
# x is from 1 to 4, containing 3 columns
# y=2 is from y-border_size, 3-1 = 2; y=5 is from y+border_size+1, 3+1+1 = 5
# x=1 is from x-border_size, 1-1 = 0; x=4 is from x+border_size+1, 1+1+1 = 4

array([[ 6,  7,  8],
       [11, 12, 13],
       [16, 17, 18]])

In [14]:
# go through ever element in k_array
for row in k_paddedled[2:5, 1:4]:
    for element in row:
        print(element, end=' ')
    print()

6 7 8 
11 12 13 
16 17 18 


In [15]:

import numpy as np

# 创建一个二维数组（例如，图像）
input_array = np.array([[1, 2, 0],
                        [5, 3, 1],
                        [0, 1, 2]])

# 创建一个卷积核
kernel = np.array([[1, 1, 1],
                   [1, 1, 1],
                   [1, 1, 1]])

# 执行卷积
output_array = convolve2d(input_array, kernel, mode='same', boundary='fill', fillvalue=0)


In [16]:

import numpy as np

# 创建一个二维数组（例如，图像）
input_array = np.array([[255, 255, 255],
                        [255, 255, 255],
                        [255, 255, 255]])

# 创建一个卷积核
kernel = np.array([[1, 1, 1],
                   [1, 1, 1],
                   [1, 1, 1]])

# 执行卷积
output_array = convolve2d(input_array, kernel, mode='same', boundary='fill', fillvalue=0)


In [17]:
k_paddedled

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4,  5,  0],
       [ 0,  6,  7,  8,  9, 10,  0],
       [ 0, 11, 12, 13, 14, 15,  0],
       [ 0, 16, 17, 18, 19, 20,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [18]:
# rotate the kernel by 90 degrees counterclockwise
print(np.rot90(kernel, 2))

[[1 1 1]
 [1 1 1]
 [1 1 1]]


In [19]:
output_array = convolve2d(input_array, kernel, mode='same', boundary='fill', fillvalue=0)

In [20]:
output_array

array([[1020, 1530, 1020],
       [1530, 2295, 1530],
       [1020, 1530, 1020]])